# Simulation of Molecules using quantum computing

In [ ]:
import numpy as numpy
numpy.random.seed(999999)
target_distr = numpy.random.rand(2)
target_distr /= sum(target_distr)

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [ ]:
from qiskit import Aer, transpile, assemble
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probab_distri(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return output_distr

def objective_function(params):
    qc = get_var_form(params)
    t_qc = transpile(qc, backend)
    qobj = assemble(t_qc, shots=NUM_SHOTS)
    result = backend.run(qobj).result()
    output_distr = get_probab_distri(result.get_counts(qc))
    cost = sum([numpy.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

In [ ]:
from qiskit.utils import COBYLA

optimizer = COBYLA(maxiter=500, tol=0.0001)

params = numpy.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

qc = get_var_form(ret[0])
t_qc = transpile(qc, backend)
qobj = assemble(t_qc, shots=NUM_SHOTS)
counts = backend.run(qobj).result().get_counts(qc)
output_distr = get_probab_distri(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])


Traceback (most recent call last):
  Input In [4] in <cell line: 1>
    from qiskit.aqua.components.optimizers import COBYLA
ModuleNotFoundError: No module named 'qiskit.aqua'

Use %tb to get the full traceback.
